In [1]:
! pip install kss konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.4 MB 1.3 MB/s 
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 131 kB 64.9 MB/s 
     |████████████████████████████████| 453 kB 45.1 MB/s 
  Created wheel for kss: filename=kss-3.4.3-py3-none-any.whl size=42448068 sha256=6331e1af741d91a3fc59134d639f1071b1139e103bae2bb500ba59ab49679b72
  Stored in directory: /root/.cache/pip/wheels/10/df/7d/0d0a58843aa7d267687671cf57274080bcfe8f79d59ed8f399
Successfully built kss


In [2]:
import nltk, kss, konlpy

In [14]:
t_data = '나는 오늘은 습기가 높아서 슬퍼 강의는 잘 듣고 있니?'

In [4]:
from konlpy.tag import Okt
tk = Okt() #쪼개기만 하고 vocabulary를 만들어주진 않는다

In [6]:
tk.morphs(t_data)

['오늘', '은', '습기', '가', '높아서', '슬퍼', '강의', '는', '잘', '듣고', '있니', '?']

In [15]:
def 문장_처리(in_t):
    입력문자 = in_t.replace('?','') #단어의 관점에선 ?가 의미를 가지지 않으므로 없애버린다
    토큰_data = tk.morphs(입력문자)
    단어집합={}
    단어빈도수=[]
    for 단어 in 토큰_data:
        if 단어 not in 단어집합:
            단어집합[단어] = len(단어집합) #단어집합 길이만큼 하면 해당 단어가 인덱스처럼 된다.
            단어빈도수.insert(len(단어집합)-1,1)
        else:
            단어인덱스=단어집합[단어]
            단어빈도수[단어인덱스] += 1
    return 단어집합, 단어빈도수
문장_처리(t_data)

({'가': 5,
  '강의': 8,
  '나': 0,
  '높아서': 6,
  '는': 1,
  '듣고': 10,
  '슬퍼': 7,
  '습기': 4,
  '오늘': 2,
  '은': 3,
  '있니': 11,
  '잘': 9},
 [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [16]:
from sklearn.feature_extraction.text import CountVectorizer #BoW를 만드려 함
t_t = t_data.replace('?', '')
bv = CountVectorizer()
bv.fit_transform([t_t]).toarray()

array([[1, 1, 1, 1, 1, 1, 1, 1]])

In [17]:
bv.vocabulary_

{'강의는': 0, '나는': 1, '높아서': 2, '듣고': 3, '슬퍼': 4, '습기가': 5, '오늘은': 6, '있니': 7}

In [18]:
문서=['나는 강의는 어려워','나는 날씨가 덥고 습기가 많아','나는 오늘은 그냥 있니?', '나는 수업은 듣고 있니?']

In [19]:
bv.transform(문서).toarray()

array([[1, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 1, 0, 0, 0, 1]])

In [20]:
문서

['나는 강의는 어려워', '나는 날씨가 덥고 습기가 많아', '나는 오늘은 그냥 있니?', '나는 수업은 듣고 있니?']

In [21]:
bv2=CountVectorizer(stop_words=['나는'])
bv2.fit_transform(문서).toarray()

array([[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]])

In [22]:
bv2.vocabulary_

{'강의는': 0,
 '그냥': 1,
 '날씨가': 2,
 '덥고': 3,
 '듣고': 4,
 '많아': 5,
 '수업은': 6,
 '습기가': 7,
 '어려워': 8,
 '오늘은': 9,
 '있니': 10}

TF - IDF 

In [30]:
t_data = [
    '공부 하고 싶다',
    '공부 하기 싫다',
    '공부 내일 할까',
    '공부 하고 놀기',
    '놀기 말고 놀기'
]

from math import log
import pandas as pd


In [31]:
vos = list(set(단어 for 문장 in t_data for 단어 in 문장.split()))
vos

['싶다', '하고', '내일', '싫다', '할까', '하기', '놀기', '말고', '공부']

In [25]:
# vos2 = list(set(단어 for 단어 in (문장.split() for 문장 in t_data)))
# vos2

TypeError: ignored

In [32]:
n = len(t_data)
def tf(t,d):
    return d.count(t)

def idf(t):
    df=0
    for 문장 in t_data:
        df += t in 문장 #boolean값도 수처럼 취급. True는 1 False는 0
    return log(n/(df+1))
def tfidf(t,d):
    return tf(t,d) * idf(t)

DTM

In [33]:
t_l = []
for i in range(n):
    t_l.append([])
    d=t_data[i]
    for j in range(len(vos)):
        t = vos[j]
        t_l[i].append(tf(t,d))
tf_ = pd.DataFrame(t_l, columns=vos)
tf_

,싶다,하고,내일,싫다,할까,하기,놀기,말고,공부
0,1,1,0,0,0,0,0,0,1
1,0,0,0,1,0,1,0,0,1
2,0,0,1,0,1,0,0,0,1
3,0,1,0,0,0,0,1,0,1
4,0,0,0,0,0,0,2,1,0


In [34]:
idf_l = []
for i in range(len(vos)):
    t=vos[i]
    idf_l.append(idf(t))
idf_ = pd.DataFrame(idf_l,index=vos, columns=['IDF'])
idf_

,IDF
싶다,0.916291
하고,0.510826
내일,0.916291
싫다,0.916291
할까,0.916291
하기,0.916291
놀기,0.510826
말고,0.916291
공부,0.000000


In [35]:
tf_idf_l =[]
for i in range(n):
    tf_idf_l.append([])
    d = t_data[i]
    for j in range(len(vos)):
        t = vos[j]
        tf_idf_l[i].append(tfidf(t,d))
tf_idf_ = pd.DataFrame(tf_idf_l,columns=vos)
tf_idf_

,싶다,하고,내일,싫다,할까,하기,놀기,말고,공부
0,0.916291,0.510826,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.000000,0.000000,0.000000,0.916291,0.000000,0.916291,0.000000,0.000000,0.0
2,0.000000,0.000000,0.916291,0.000000,0.916291,0.000000,0.000000,0.000000,0.0
3,0.000000,0.510826,0.000000,0.000000,0.000000,0.000000,0.510826,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.021651,0.916291,0.0


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
t_data

['공부 하고 싶다', '공부 하기 싫다', '공부 내일 할까', '공부 하고 놀기', '놀기 말고 놀기']

In [37]:
tfidfv = TfidfVectorizer().fit(t_data)
tfidfv.vocabulary_

{'공부': 0,
 '내일': 1,
 '놀기': 2,
 '말고': 3,
 '싫다': 4,
 '싶다': 5,
 '하고': 6,
 '하기': 7,
 '할까': 8}

In [38]:
tfidfv.transform(t_data).toarray()

array([[0.40156512, 0.        , 0.        , 0.        , 0.        ,
        0.71277522, 0.57506256, 0.        , 0.        ],
       [0.37008621, 0.        , 0.        , 0.        , 0.65690037,
        0.        , 0.        , 0.65690037, 0.        ],
       [0.37008621, 0.65690037, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.65690037],
       [0.44274009, 0.        , 0.63402729, 0.        , 0.        ,
        0.        , 0.63402729, 0.        , 0.        ],
       [0.        , 0.        , 0.85000275, 0.52677825, 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

코사인 유사도

In [39]:
import numpy as np
data1=np.array([1,1,1,1])
np.linalg.norm(data1) #코사인 유사도의 분모 공식

2.0

In [40]:
def cos_i(A,B):
    return A.dot(B)/(np.linalg.norm(A) * np.linalg.norm(B)) #벡터 간의 내적 / 스칼라 곱(그냥 곱)


In [42]:
d1=np.array([0,5,5,5])
d2=np.array([0,-1,-1,-1])
cos_i(d1, d2)

-1.0

In [44]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
data = pd.read_csv('data1.csv', low_memory=False)
data = data.head(20000)
data['overview'] = data['overview'].fillna('')

In [45]:
tfidf=TfidfVectorizer(stop_words='english')
tfidf_t=tfidf.fit_transform(data['overview'])
tfidf_t

<20000x47487 sparse matrix of type '<class 'numpy.float64'>'
	with 535909 stored elements in Compressed Sparse Row format>

In [46]:
#각각의 문장에 대해 서로간의 유사도가 생김
cos = cosine_similarity(tfidf_t, tfidf_t)
cos.shape

(20000, 20000)

In [47]:
data.index

RangeIndex(start=0, stop=20000, step=1)

In [ ]:
#유사한 영화를 추천하기 위해 title과 인덱스를 연계함. 컴퓨터가 알아먹을 수 있도록
t_idx = dict(zip(data['title'], data.index)) # title에 대한 vocabulary
#t_idx

In [57]:
def ck_s_t(t,cosine_sim = cos):
    idx=t_idx[t]
    c_sc = list(enumerate(cosine_sim[idx]))
    c_sc = sorted(c_sc, key = lambda x: x[1], reverse=True)
    m_d = c_sc[1:6]  #5개만 보기
    m_i = [i[0] for i in m_d]  #그 값들의 인덱스
    return data['title'].iloc[m_i] #인덱스에 해당하는 제목

In [58]:
ck_s_t('Toy Story')

15348               Toy Story 3
2997                Toy Story 2
10301    The 40 Year Old Virgin
8327                  The Champ
1071      Rebel Without a Cause
Name: title, dtype: object

자카드 유사도

In [60]:
A = set([1,2,3,4,5])
B = set([4,5,6,7,8,9])
A | B #합집합

{1, 2, 3, 4, 5, 6, 7, 8, 9}

In [62]:
A & B #교집합

{4, 5}

In [64]:
data1 = '나는 오늘 정말 피곤해 그래서 힘들어'.split()
data2 = '나는 오늘 정말 못해 그래서 힘들어'.split()
data3 = '나는 오늘 너무 즐거워 그래서 행복해'.split()

In [65]:
un = set(data1) | set(data2)
un

{'그래서', '나는', '못해', '오늘', '정말', '피곤해', '힘들어'}

In [66]:
intd = set(data1) & set(data2)
intd

{'그래서', '나는', '오늘', '정말', '힘들어'}

In [67]:
len(intd) / len(un)

0.7142857142857143

In [68]:
un1 = set(data1) | set(data3)
intd1 = set(data1) & set(data3)
len(intd1) / len(un1)
#data1, data2 간의 유사도가 data1, data3 간의 유사도보다 높다고 볼 수 있다.

0.3333333333333333

유클리디안 유사도

In [70]:
data1 = '나는 오늘 정말 피곤해 그래서 힘들어'
data2 = '나는 오늘 정말 못해 그래서 힘들어'
data3 = '나는 오늘 너무 즐거워 그래서 행복해'

In [71]:
t_v = TfidfVectorizer() #tfidf로 벡터화한 후 유클리디안 유사도를 판별하려 한다.
m_data = t_v.fit_transform([data1, data2, data3])
m_data.toarray()

array([[0.32999531, 0.32999531, 0.        , 0.        , 0.32999531,
        0.42492904, 0.        , 0.55873062, 0.        , 0.42492904],
       [0.32999531, 0.32999531, 0.        , 0.55873062, 0.32999531,
        0.42492904, 0.        , 0.        , 0.        , 0.42492904],
       [0.29360705, 0.29360705, 0.49711994, 0.        , 0.29360705,
        0.        , 0.49711994, 0.        , 0.49711994, 0.        ]])

In [76]:
def f(A,B):
    return np.sqrt(np.sum((A-B)**2))

In [74]:
A = m_data.toarray()[0]
B = m_data.toarray()[1]
C = m_data.toarray()[2]

In [78]:
f(A,B),f(A,C)

(0.7901644267844229, 1.1910777960950294)

In [82]:
def e_f(m): #정규화
    return m/np.sum(m)
e_m_d = e_f(m_data)

In [84]:
from sklearn.metrics import euclidean_distances
euclidean_distances(e_m_d[0], e_m_d[1]), euclidean_distances(e_m_d[0], e_m_d[2])

(array([[0.11021454]]), array([[0.16613516]]))

멘하탄 거리 유사도

In [85]:
from sklearn.metrics.pairwise import manhattan_distances
print(manhattan_distances(m_data[0], m_data[1]), manhattan_distances(m_data[0], m_data[2]))
print(manhattan_distances(e_m_d[0], e_m_d[1]), manhattan_distances(e_m_d[0], e_m_d[2]))

[[1.11746125]] [[3.00911332]]
[[0.1558669]] [[0.41972028]]


In [89]:
data = [data1, data2, data3]
from tensorflow.keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(data)
tk.word_index

{'그래서': 3,
 '나는': 1,
 '너무': 8,
 '못해': 7,
 '오늘': 2,
 '정말': 4,
 '즐거워': 9,
 '피곤해': 6,
 '행복해': 10,
 '힘들어': 5}

In [88]:
tk.texts_to_matrix(data, mode='count')

array([[0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1.]])

In [90]:
tk.texts_to_matrix(data, mode='tfidf')

array([[0.        , 0.55961579, 0.55961579, 0.55961579, 0.69314718,
        0.69314718, 0.91629073, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.55961579, 0.55961579, 0.55961579, 0.69314718,
        0.69314718, 0.        , 0.91629073, 0.        , 0.        ,
        0.        ],
       [0.        , 0.55961579, 0.55961579, 0.55961579, 0.        ,
        0.        , 0.        , 0.        , 0.91629073, 0.91629073,
        0.91629073]])

1. data 로드를 통한 데이터 프레임 완성
2. 한글에 맞게 형태소 분석기를 이용해 단어 토큰화 TF IDF 계산
3. 코사인 유사도를 기반으로 영화 3종 추천
-올드보이 제시-

In [91]:
data = pd.read_csv('data2.csv')
data

In [112]:
data

,name,genre,content
0,올드보이,스릴러,술 좋아하고 떠들기 좋아하는 오.대.수. 본인의 이름풀이를 오늘만 대충 수습하며 살...
1,노트북,로맨스,"17살, ‘노아’는 밝고 순수한 ‘앨리’를 보고 첫눈에 반한다. 빠른 속도로 서로에..."
2,미션임파서블,액션,"도난당한 핵무기를 되찾아 오는 위험천만한 작전 중, 에단 헌트(톰 크루즈)는 임무를..."
3,니모를찾아서,애니메이션,“디즈니`픽사 최고의 애니메이션이 3D로 다시 찾아온다!” 호기심 가득한 아기 물고...
4,탑건,액션,해군 최신 전투기 F-14기를 모는 젊은 조종사 매버릭 대위(톰 크루즈)는 최고의 ...
5,친절한금자씨,스릴러,주변 사람들의 시선을 단번에 사로잡을 만큼 뛰어난 미모의 소유자인 금자(이영애)는 ...
6,반지의제왕,판타지,모든 힘을 지배할 악의 군주 ‘사우론’의 절대반지가 깨어나고 악의 세력이 세상을 지...
7,신세계,범죄,경찰청 수사 기획과 강과장(최민식)은 국내 최대 범죄 조직인 '골드문'이 기업형 조...
8,아저씨,NaN,불행한 사건으로 아내를 잃고 세상을 등진 채 전당포를 꾸려가며 외롭게 살아가는 전직...
9,해리포터,NaN,해리 포터(다니엘 래드클리프 분)는 위압적인 버논 숙부(리챠드 그리피스 분)와 냉담...


In [ ]:
review = data['content']
review

In [96]:
from konlpy.tag import Okt
tg = Okt()


In [ ]:
tg.morphs(review[0])

In [ ]:
t_reviews = []
for i in review:
    tk_r =' '.join(tg.morphs(i))
    t_reviews.append(tk_r)
t_reviews

In [106]:
t_reviews[0]

"술 좋아하고 떠들기 좋아하는 오 . 대 . 수 . 본인 의 이름 풀이 를 오늘 만 대충 수습 하며 살자 ' 라고 이죽거리는 이 남자 는 아내 와 어린 딸아이 를 가진 지극히 평범한 샐러리맨 이다 . 어느 날 , 술 이 거나 하게 취해 집 에 돌아가는 길 에 존재 를 알 수 없는 누군가 에게 납치 , 사설 감금 방 에 갇히게 되는데 . 언뜻 보면 싸구려 호텔 방 을 연상 케 하는 감금 방 . 중국집 군 만두 만을 먹으며 8 평이 라는 제한 된 공간 에서 그 가 할 수 있는 일이 라곤 , 텔레비전 보는 게 전부 . 그렇게 1년 이 지났을 무렵 , 뉴스 를 통해 나오는 아내 의 살해 소식 . 게다가 아내 의 살인 범 으로 자신 이 지목 되고 있 음 을 알 게 된 오 대수 는 자살 을 감행 하 지만 죽는 것 조차 그 에겐 용납 되지 않는다 . 오 대수 는 복수 를 위해 체력 단련 을 비롯 , 자신 을 가둘만 한 사람 들 , 사건 들 을 모조리 기억 속 에서 꺼내 악행 의 자서전 을 기록 한다 . 한편 , 탈출 을 위해 감금 방 한쪽 구석 을 쇠 젓가락 으로 파기 도 하는데 .. 감금 15년 을 맞이 하는 해 , 마침내 사람 몸 하나 빠져나갈 만큼의 탈출 구가 생겼을 때 , 어이없게도 15년 전 납치 됐던 바로 그 장소 로 풀려나 있는 자신 을 발견 하게 된다 . 우연히 들른 일식집 에서 갑자기 정신 을 잃어버린 오 대수 는 보조 요리사 미도 집 으로 가게 되고 , 미도 는 오 대수 에게 연민 에서 시작 한 사랑 의 감정 을 키워 나가게 된다 . 한편 감금 방 에서 먹던 군 만두 에서 나온 청룡 이란 전표 하나로 찾아낸 7.5 층 감금 방 의 정체 를 찾아내고 ... 마침내 , 첫 대면 을 하는 날 복수심 으로 들끓는 대수 에게 우진 은 너무나 냉정하게 게임 을 제안 한다 . 자신 이 가둔 이유 를 5일 안 에 밝혀내면 스스로 죽어주겠다는 것 . 대수 는 이 지독한 비밀 을 풀기 위해 , 사랑 하는 연인 , 미도 를 잃지 않기 위해 5일 간의 긴박한 수수께끼 를 풀어

In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_ = TfidfVectorizer()
tfidf_.fit(t_reviews)

TfidfVectorizer()

In [ ]:
tfidf_.transform(t_reviews).toarray()

In [109]:
tf_idf = tfidf_.transform(t_reviews)

In [111]:
from sklearn.metrics.pairwise import cosine_similarity
cos_ = cosine_similarity(tf_idf,tf_idf)

In [114]:
t_idx = dict(zip(data['name'], data.index))

In [ ]:
# def ck_s_t(t,cosine_sim = cos):
#     idx=t_idx[t]
#     c_sc = list(enumerate(cosine_sim[idx]))
#     c_sc = sorted(c_sc, key = lambda x: x[1], reverse=True)
#     m_d = c_sc[1:6]  #5개만 보기
#     m_i = [i[0] for i in m_d]  #그 값들의 인덱스
#     return data['title'].iloc[m_i] #인덱스에 해당하는 제목

In [115]:
def rec(name, cos=cos_):
    idx = t_idx[name]
    c_sc = list(enumerate(cos[idx]))
    c_sc = sorted(c_sc, key = lambda x: x[1], reverse = True)
    name3 = c_sc[1:4]
    name3_i = [i[0] for i in name3]
    return data['name'].iloc[name3_i]

In [116]:
rec('올드보이')

5     친절한금자씨
11    어바웃 타임
8        아저씨
Name: name, dtype: object

In [117]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import euclidean_distances
tfidf_v = TfidfVectorizer()
tfidf_m = tfidf_v.fit_transform(t_reviews)

In [118]:
cos = cosine_similarity(tfidf_m, tfidf_m)
eud = euclidean_distances(tfidf_m, tfidf_m)
cos.shape, eud.shape

((18, 18), (18, 18))

In [125]:
t_idx = dict(zip(data['name'], data.index))
def ck_s_t(t, cosine_sim=cos, k=0):
    idx = t_idx[t]
    c_sc = list(enumerate(cosine_sim[idx]))
    if k == 0:
        c_sc=sorted(c_sc, key=lambda x: x[1], reverse=True)
    else:
        c_sc=sorted(c_sc, key=lambda x: x[1])
    m_i = c_sc[1:5]
    m_idx = [i[0] for i in m_i]
    return data['name'].loc[m_idx]
ck_s_t('올드보이', k=0)

5       친절한금자씨
11      어바웃 타임
8          아저씨
14    지금만나러갑니다
Name: name, dtype: object